In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/github_repositories/my_repositories/SteadyStateSchrodingerEquation`


In [2]:
#= Pkg.add("Gridap");
Pkg.add("GridapGmsh");
Pkg.add("Gmsh");
Pkg.add("FileIO");
Pkg.add("LinearAlgebra");
Pkg.add("SparseArrays");
Pkg.add("Arpack"); =#

In [3]:
#= Pkg.status() =#

In [4]:
using Gridap
using GridapGmsh
using Gmsh
using FileIO
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Arpack

In [5]:
include("../src/SteadyStateSchrodingerEquation.jl")

Main.SteadyStateSchrodingerEquation

In [6]:
using .SteadyStateSchrodingerEquation

### Chequeamos functiones dentro de `MeshGeneratorFunction.jl`

In [7]:
dom2D=(-1.0,1.0,-1.0,1.0);nxy=(10,10);params_model=((dom2D,nxy));
grid_type="Cartesian2D";

In [8]:
model2D=make_model(grid_type,params_model);

In [9]:
#= using Gridap
writevtk(model2d,"hola") =#

### Chequeamos funciones dentro de `BoundaryConditionsFunction.jl`

In [10]:
BC_type="FullDirichlet";
FullDirichlet_values,FullDirichlet_tags=make_boundary_conditions(grid_type,BC_type,ComplexF64);

### Chequeamos funciones dentro de `MiscellaneousFunctions.jl`

In [11]:
Ω,dΩ,Γ,dΓ = measures(model2D,3,FullDirichlet_tags)

(BodyFittedTriangulation(), GenericMeasure(), BoundaryTriangulation(), GenericMeasure())

In [12]:
reff = ReferenceFE(lagrangian,Float64,2)

(Lagrangian(), (Float64, 2), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [13]:
VSpace,USpace = FESpaces(model2D,reff,grid_type;BC_type=BC_type,TypeData=ComplexF64)

(UnconstrainedFESpace(), TrialFESpace())

Construimos funciones para escribir una formulación de tipo Sturm-Liouville particular

In [14]:
using Printf

In [21]:
const ħ=1.0;
const m=1.0;
function eigenvalue_problem_functions(params;switch_potential = "QHO_1D")
    if (switch_potential == "QHO_1D")
        # caso de potencial tipo quantum harmonic oscillator 1D (QHO)
        @printf("Set quantum harmonic oscillator 1D potential\n");
        ω,x₁=params;
        pₕ_QHO_1D(x) = 0.5*(ħ*ħ)*(1.0/m);                                      # factor para energía cinética
        qₕ_QHO_1D(x) = 0.5*m*(ω*ω)*(x[1]-x₁)*(x[1]-x₁);                        # oscilador armónico 1D centrado en x₁
        rₕ_QHO_1D(x) = 1.0;
        return pₕ_QHO_1D,qₕ_QHO_1D,rₕ_QHO_1D;
    elseif (switch_potential == "QHO_2D")
        # caso de potencial tipo quantum harmonic oscillator 2D (QHO)
        @printf("Set quantum harmonic oscillator 2D potential\n");
        ω,x₁,y₁=params;
        pₕ_QHO_2D(x) = 0.5*(ħ*ħ)*(1.0/m);                                       # factor para energía cinética
        qₕ_QHO_2D(x) = 0.5*m*(ω*ω)*((x[1]-x₁)*(x[1]-x₁)+(x[2]-y₁)*(x[2]-y₁));   # oscilador armónico 2D centrado en (x₁,y₁)
        rₕ_QHO_2D(x) = 1.0;
        return pₕ_QHO_2D,qₕ_QHO_2D,rₕ_QHO_2D;
    elseif (switch_potential == "FWP")
        # caso de potencial tipo finite well potential (FWP)
        @printf("Set quantum finite well potential\n");
        V₀_FWP,a_FWP=params;
        pₕ_FWP(x) = 0.5*(ħ*ħ)*(1.0/m);                                          # factor para energía cinética
        qₕ_FWP(x) = interval.(x[1],-a_FWP,a_FWP,V₀_FWP)
        rₕ_FWP(x) = 1.0;
        return pₕ_FWP,qₕ_FWP,rₕ_FWP;
    elseif (switch_potential == "Electron_Nuclear_Potential_1D")
        # caso de potencial tipo interacción electron-nucleo en pozo nuclear
        @printf("Set Electron-Nuclear potential with fixed R\n");
        R,R₁,R₂,Rc,Rf=params;
        pₕ_ENP_1D(x) = 0.5*(ħ*ħ)*(1.0/m);                                          # factor para energía cinética
        qₕ_ENP_1D(x) = CoulombPotential(R,R₁)+CoulombPotential(R,R₂)+
            Aprox_Coulomb_Potential(x[1],R₁,Rf)+Aprox_Coulomb_Potential(x[1],R,Rc)+Aprox_Coulomb_Potential(x[1],R₂,Rf)
        rₕ_ENP_1D(x) = 1.0;
        return pₕ_ENP_1D,qₕ_ENP_1D,rₕ_ENP_1D;
    elseif (switch_potential == "Electron_Nuclear_Potential_2D")
        # caso de potencial tipo interacción electron-nucleo en pozo nuclear
        @printf("Set Electron-Nuclear potential\n");
        R₁,R₂,Rc,Rf=params;
        pₕ_ENP_2D(x) = 0.5*(ħ*ħ)*(1.0/m);     # factor para energía cinética
        qₕ_ENP_2D(x) = CoulombPotential(x[2]*(1.0/γ),R₁)+CoulombPotential(x[2]*(1.0/γ),R₂)+
            Aprox_Coulomb_Potential(x[1],R₁,Rf)+Aprox_Coulomb_Potential(x[1],x[2]*(1.0/γ),Rc)+Aprox_Coulomb_Potential(x[1],R₂,Rf)
        rₕ_ENP_2D(x) = 1.0;
        return pₕ_ENP_2D,qₕ_ENP_2D,rₕ_ENP_2D;
    end
end

eigenvalue_problem_functions (generic function with 1 method)

In [22]:
p,q,r = eigenvalue_problem_functions((1.0,0.0,0.0);switch_potential = "QHO_2D")

Set quantum harmonic oscillator 2D potential


(var"#pₕ_QHO_2D#47"(), var"#qₕ_QHO_2D#48"{Float64}(0.0, Core.Box(0.0), Core.Box(1.0)), var"#rₕ_QHO_2D#49"())

### Chequeamos funciones dentro de `EigenProblemSolveFunction.jl`

In [23]:
ϵ,ϕ = EigenValuesAndEigenVectors(p,q,r,dΩ,USpace,VSpace)

(ComplexF64[2.5969681919949563 + 6.970520979061812e-17im, 6.376116799836291 + 1.801837033651666e-16im, 6.376116799836299 - 2.4687596813292126e-16im, 10.155265407677616 - 2.2007814920120522e-16im, 12.580402282186753 - 1.6538989482112496e-16im, 12.580402282186798 - 6.106831889442633e-16im, 16.35955089002805 - 2.426519582772567e-16im, 16.359550890028117 - 2.8633558672169306e-16im, 21.324125206189777 + 2.5466631674742934e-16im, 21.324125206189848 + 7.613474859600007e-16im], CellField[SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction()])